# Welcome to PYNQ

## Getting Started

To get started using PYNQ, try running the example notebooks in the folders described below. 

* **getting_started**: includes an introduction to using Jupyter notebook with PYNQ, the Python environment, and how to use some basic features of the curernt platform. 

* **common**: contains example notebooks on how to download an overlay, how to set the Zynq clocks, how to execute Linux shell commands, and how to use USB devices.

If other overlays or packages are installed, other folders with example notebooks may also be available in this directory.  


## Documentation

Please see the latest <a href="http://pynq.readthedocs.io">PYNQ Documentation on readthedocs</a>.  


## Support

For questions or support, go to the forum on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## Project webpage

You can find details on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## GitHub

The PYNQ Repository is hosted on github: <a href="https://github.com/Xilinx/PYNQ">PYNQ GitHub Repository</a>.

In [1]:
import os

In [2]:
from pynq import Overlay

In [3]:
os.getcwd()

'/home/xilinx/jupyter_notebooks'

In [4]:
# Use bitstream for PL 
overlay = Overlay(os.getcwd() + "/fifo_example/fifo_1d/fifo.bit")

In [7]:
overlay?

In [8]:
# 1D FIFO
fifo_ip = overlay.fifo
dma = overlay.fifo_dma
fifo_ip?

In [9]:
# View Registers
fifo_ip.register_map


RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  size = Register(size=0),
  size_ctrl = Register(size_ap_vld=0, RESERVED=0)
}

In [10]:
help(fifo_ip.register_map.CTRL)

Help on RegisterCTRL in module pynq.registers object:

class RegisterCTRL(Register)
 |  Control signals
 |  
 |  Method resolution order:
 |      RegisterCTRL
 |      Register
 |      builtins.object
 |  
 |  Data descriptors defined here:
 |  
 |  AP_START
 |      Control signals
 |  
 |  AP_DONE
 |      Control signals
 |  
 |  AP_IDLE
 |      Control signals
 |  
 |  AP_READY
 |      Control signals
 |  
 |  RESERVED_1
 |      Control signals
 |  
 |  AUTO_RESTART
 |      Control signals
 |  
 |  RESERVED_2
 |      Control signals
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  _fields = {'AP_DONE': {'access': 'read-only', 'bit_offset': 1, 'bit_wi...
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Register:
 |  
 |  __getitem__(self, index)
 |      Get the register value.
 |      
 |      This method accepts both integer index, or slice as i

In [11]:
import numpy as np
from pynq import allocate

In [12]:
# Will run for one intake
# Enable AUTO_RESTART for continuous runs
fifo_ip.register_map.CTRL.AP_START = 1

In [13]:
# Read numpy array size
fifo_ip.register_map.size

Register(size=10)

In [14]:
# Prove size is read-only
fifo_ip.register_map.size = 1000

AttributeError: can't set attribute

In [15]:
size = int(fifo_ip.register_map.size)
print(size)

10


In [16]:
# size = 10
inStream = allocate(shape=(size,), dtype=np.int32)
outStream = allocate(shape=(size,), dtype=np.int32)

for i in range(size):
    inStream[i] = 100
    
inStream.flush()
print(inStream)
print(outStream)


[100 100 100 100 100 100 100 100 100 100]
[0 0 0 0 0 0 0 0 0 0]


In [17]:
dma.sendchannel.transfer(inStream)
print("Completed transfer inStream")



Completed transfer inStream


In [18]:
dma.recvchannel.transfer(outStream)
print("Completed transfer outStream")

Completed transfer outStream


In [19]:
# Call if TLAST is not set high in HLS code
dma.sendchannel.wait()
print("Completed wait send")


Completed wait send


In [20]:
# Call if TLAST is not set high in HLS code
dma.recvchannel.wait()
print("Completed wait recv")

Completed wait recv


In [21]:
print(outStream)

[100 100 100 100 100 100 100 100 100 100]


In [22]:
inStream.freebuffer()
print("Freed inStream buffer")

Freed inStream buffer


In [23]:
outStream.freebuffer()
print("Freed outStream buffer")

Freed outStream buffer


In [24]:
fifo_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  size = Register(size=10),
  size_ctrl = Register(size_ap_vld=1, RESERVED=0)
}